### Import library

In [28]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR100
transform = transforms.ToTensor()

# create training dataset
trainset = CIFAR100(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [47]:

print(idx)
print(trainset[0])


tensor([False, False,  True,  ..., False, False, False])
(tensor([[[1.0000, 1.0000, 1.0000,  ..., 0.7647, 0.8314, 0.7137],
         [1.0000, 0.9961, 0.9961,  ..., 0.6667, 0.6314, 0.5725],
         [1.0000, 0.9961, 1.0000,  ..., 0.7412, 0.6510, 0.4745],
         ...,
         [0.5804, 0.5569, 0.5490,  ..., 0.1176, 0.2549, 0.2980],
         [0.4784, 0.4706, 0.4941,  ..., 0.0863, 0.3804, 0.5529],
         [0.3412, 0.3451, 0.3961,  ..., 0.1333, 0.4118, 0.5412]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.8039, 0.8784, 0.7608],
         [1.0000, 0.9961, 0.9961,  ..., 0.6902, 0.6588, 0.6039],
         [1.0000, 0.9961, 1.0000,  ..., 0.7804, 0.6980, 0.5216],
         ...,
         [0.7255, 0.7137, 0.7020,  ..., 0.0667, 0.2431, 0.3020],
         [0.6157, 0.6078, 0.6275,  ..., 0.0627, 0.4392, 0.6314],
         [0.4784, 0.4784, 0.5255,  ..., 0.1412, 0.5216, 0.6784]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.7569, 0.8000, 0.6549],
         [1.0000, 0.9961, 0.9961,  ..., 0.5882, 0.5098, 0.4431],


In [55]:
# We create a tensor that has `True` at an index if the sample belongs to class 1
idx0 = torch.tensor(trainset.targets) == 0
idx1 = torch.tensor(trainset.targets) == 1
idx = idx0 | idx1

#We then need to convert this into a list of indices at which we have True.
train_indices = idx.nonzero().reshape(-1)

# Create of subset
train_subset = torch.utils.data.Subset(trainset,  train_indices)

#verify
print(train_subset[505][1])





0


In [50]:
sum(torch.tensor(trainset.targets) == 1)

tensor(500)